In [64]:
import pymysql

# your username and password go here!
creds = {'user': '', 
         'pswd': ''} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [95]:
createCore = '''CREATE TABLE IF NOT EXISTS core (
                page_id INT(8) UNSIGNED NOT NULL,
                page_title VARCHAR(255) NOT NULL,
                page_is_ep TINYINT(1) UNSIGNED NOT NULL,
                page_info_pulled BIGINT UNSIGNED NOT NULL,
                page_in_train TINYINT(1) UNSIGNED NOT NULL,
                page_is_redirect TINYINT(1) UNSIGNED NOT NULL,
                page_is_new TINYINT(1) UNSIGNED NOT NULL,
                page_random DOUBLE UNSIGNED NOT NULL,
                page_touched BIGINT UNSIGNED NOT NULL,
                page_links_updated BIGINT UNSIGNED,
                page_latest INT(8) UNSIGNED NOT NULL,
                page_len INT(8) UNSIGNED NOT NULL,
                pr_type VARCHAR(255),
                pr_level VARCHAR(255),
                pr_expiry VARCHAR(20),
                PRIMARY KEY (page_id)
                ) DEFAULT CHARACTER SET utf8'''

In [96]:
cur.execute(createCore)

In [38]:
cols_for_db = ['page_id','page_title','page_is_ep','page_info_pulled','page_in_train',
              'page_is_redirect','page_is_new','page_random','page_touched','page_links_updated',
              'page_latest','page_len','pr_type','pr_level','pr_expiry']

In [11]:
import pandas as pd
epa_articles = pd.read_csv("../WikiSQL/070916_edit_protected_articles.csv")
nepa_articles = pd.read_csv("../WikiSQL/071016_non_edit_protected_articles.csv")

In [15]:
epa_articles.shape[0]

7526

In [22]:
import numpy as np
redundant = pd.merge(epa_articles, nepa_articles, how='inner', left_on="page_id", right_on="page_id")
redundantIDs = redundant['page_id'].tolist()

nepa_articles_filtered = nepa_articles.loc[~nepa_articles['page_id'].isin(redundantIDs)].copy()

In [23]:
epa_articles['page_is_ep'] = np.ones(epa_articles.shape[0])
epa_articles['page_info_pulled'] = 20160709000000

nepa_articles_filtered['page_is_ep'] = np.zeros(nepa_articles_filtered.shape[0])
nepa_articles_filtered['page_info_pulled'] = 20160710000000

In [35]:
master_articles = pd.concat([epa_articles, nepa_articles_filtered])
select_train = np.random.rand(master_articles.shape[0])
is_train = np.array(select_train > 0.25).astype(int)

0.75167154199749264

In [36]:
master_articles['page_in_train'] = is_train

In [37]:
master_articles

,page_content_model,page_counter,page_id,page_info_pulled,page_is_ep,page_is_new,page_is_redirect,page_latest,page_len,page_links_updated,...,page_title,page_touched,pr_cascade,pr_expiry,pr_id,pr_level,pr_page,pr_type,pr_user,page_in_train
0,wikitext,558,25,20160709000000,1.0,0,0,728503740,146470,2.016071e+13,...,Autism,20160705210334,0.0,infinity,373620.0,autoconfirmed,25.0,edit,NaN,1
1,wikitext,2377,290,20160709000000,1.0,0,0,725198152,19399,2.016071e+13,...,A,20160709170824,0.0,infinity,394424.0,autoconfirmed,290.0,edit,NaN,1
2,wikitext,8013,307,20160709000000,1.0,0,0,728371848,171988,2.016071e+13,...,Abraham_Lincoln,20160709073542,0.0,infinity,302458.0,autoconfirmed,307.0,edit,NaN,1
3,wikitext,4612,308,20160709000000,1.0,0,0,728907803,109151,2.016071e+13,...,Aristotle,20160708133344,0.0,infinity,314504.0,autoconfirmed,308.0,edit,NaN,1
4,wikitext,3346,594,20160709000000,1.0,0,0,728340218,126233,2.016070e+13,...,Apollo,20160704191912,0.0,infinity,523018.0,autoconfirmed,594.0,edit,NaN,1
5,wikitext,530,620,20160709000000,1.0,0,0,727823977,66673,2.016071e+13,...,Animal_Farm,20160705100358,0.0,infinity,527142.0,autoconfirmed,620.0,edit,NaN,1
6,wikitext,1527,624,20160709000000,1.0,0,0,728356845,131729,2.016071e+13,...,Alaska,20160708072835,0.0,infinity,573302.0,autoconfirmed,624.0,edit,NaN,0
7,wikitext,669,628,20160709000000,1.0,0,0,727781810,48260,2.016071e+13,...,Aldous_Huxley,20160705100358,0.0,20161225220105,625181.0,autoconfirmed,628.0,edit,NaN,1
8,wikitext,1421,655,20160709000000,1.0,0,0,715213523,43130,2.016070e+13,...,Abacus,20160704191912,0.0,20160906162351,588481.0,autoconfirmed,655.0,edit,NaN,1
9,wikitext,935,662,20160709000000,1.0,0,0,729053780,89048,2.016071e+13,...,Apollo_11,20160709150440,0.0,infinity,451504.0,autoconfirmed,662.0,edit,NaN,1


In [80]:
master_articles = master_articles.where((pd.notnull(master_articles)), None) #converts NaNs to None type, required for mySQL

,page_content_model,page_counter,page_id,page_info_pulled,page_is_ep,page_is_new,page_is_redirect,page_latest,page_len,page_links_updated,...,page_title,page_touched,pr_cascade,pr_expiry,pr_id,pr_level,pr_page,pr_type,pr_user,page_in_train
49999,wikitext,0,44237633,20160710000000,0,0,0,721089498,4063,2.01606e+13,...,The_Year_of_Billy_Miller,20160623144143,None,None,None,None,None,None,None,1


In [81]:
ma_for_db = master_articles[cols_for_db].to_dict(orient='records') #provides each row as a dict;
#keys are colnames, values are column values, the order of keys is retained from row to row

[{'page_id': 25,
  'page_in_train': 1,
  'page_info_pulled': 20160709000000,
  'page_is_ep': 1.0,
  'page_is_new': 0,
  'page_is_redirect': 0,
  'page_latest': 728503740,
  'page_len': 146470,
  'page_links_updated': 20160705210338.0,
  'page_random': 0.626026654267708,
  'page_title': 'Autism',
  'page_touched': 20160705210334,
  'pr_expiry': 'infinity',
  'pr_level': 'autoconfirmed',
  'pr_type': 'edit'},
 {'page_id': 290,
  'page_in_train': 1,
  'page_info_pulled': 20160709000000,
  'page_is_ep': 1.0,
  'page_is_new': 0,
  'page_is_redirect': 0,
  'page_latest': 725198152,
  'page_len': 19399,
  'page_links_updated': 20160709170532.0,
  'page_random': 0.8541802650822142,
  'page_title': 'A',
  'page_touched': 20160709170824,
  'pr_expiry': 'infinity',
  'pr_level': 'autoconfirmed',
  'pr_type': 'edit'},
 {'page_id': 307,
  'page_in_train': 1,
  'page_info_pulled': 20160709000000,
  'page_is_ep': 1.0,
  'page_is_new': 0,
  'page_is_redirect': 0,
  'page_latest': 728371848,
  'page_le

In [57]:
cur.execute("SET autocommit=0") #makes batch inserts SO MUCH EASIER

0

In [71]:
#generates the template query for insertions into db
insertQ = "INSERT IGNORE INTO core({}) VALUES ({})"
cols = ",".join(list(ma_for_db[0].keys()))
val_format = ",".join(['%s'] * len(list(ma_for_db[0].keys())))
formatQ = insertQ.format(cols, val_format)
formatQ

'INSERT IGNORE INTO core(pr_level,page_is_new,page_title,page_random,page_latest,page_is_ep,page_is_redirect,page_links_updated,pr_expiry,page_len,page_id,page_in_train,pr_type,page_info_pulled,page_touched) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

In [97]:
for record in ma_for_db:
    values = list(record.values())
    cur.execute(formatQ,values)

connection.commit()

In [98]:
interestingCols = ['page_id','page_title','page_is_ep','page_touched']
interestingQ = "SELECT {} FROM core"
colNameString = ",".join(interestingCols)
preppedIQ = interestingQ.format(colNameString)

cur.execute(preppedIQ)
rows = cur.fetchall() #returns tuple of tuples, preserving column order

easyPeasy = pd.DataFrame(list(rows), columns=interestingCols)
easyPeasy

,page_id,page_title,page_is_ep,page_touched
0,25,Autism,1,20160705210334
1,290,A,1,20160709170824
2,307,Abraham_Lincoln,1,20160709073542
3,308,Aristotle,1,20160708133344
4,594,Apollo,1,20160704191912
5,620,Animal_Farm,1,20160705100358
6,624,Alaska,1,20160708072835
7,628,Aldous_Huxley,1,20160705100358
8,655,Abacus,1,20160704191912
9,662,Apollo_11,1,20160709150440


In [99]:
cur.close()
connection.close()